# PEC Monte Carlo simulation

Start by importing MagmaPEC and MagmaPandas and any other packages you want to use. Here we also import Pandas for importing pressure data. For details on the use of MagmaPandas, please see it's [documentation](https://magmapandas.readthedocs.io/en/latest/).

In [1]:
import MagmaPEC as mpc
import MagmaPandas as mp

import pandas as pd

Confirm the model and PEC configurations. If you want to change models or PEC settings, follow the [configuration example](https://magmapec.readthedocs.io/en/latest/notebooks/config.html).

In [2]:
print(mpc.model_configuration)
print(mpc.PEC_configuration)


################## MagmaPandas ###################
##################################################
General settings__________________________________
fO2 buffer.....................................QFM
ΔfO2.............................................1
Melt Fe3+/Fe2+.............................sun2024
Kd Fe-Mg ol-melt........................toplis2005
Melt thermometer....................putirka2008_15
Volatile solubility model.......iaconomarziano2012
Volatile species.............................mixed
##################################################


############ Post-entrapment crystallisation ############
################### correction model ####################
Settings_________________________________________________
Fe2+ behaviour...................................buffered
Stepsize equilibration (moles)...................0.002   
Stepsize crystallisation (moles).................0.05    
Decrease factor..................................5       
FeO convergence (wt. %).......

In the next few steps we import all relevant data and set up the melt initial FeO prediction model. These steps are identical to the [FeOi](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html#) and [PEC correction](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html#) examples

Import melt inclusion and olivine data:

In [3]:
melt_file = "./data/melt.csv"
olivine_file = "./data/olivine.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
olivine = mp.read_olivine(olivine_file, index_col=["name"])

Import inclusion internal pressures or calculate them if you have measured melt CO2 (and H2O). See the [PEC model example](https://magmapec.readthedocs.io/en/latest/notebooks/pec_corr.html) for details on how to do the calculation. Here we import them from a file.

In [4]:
pressure_file ="./data/pressure.csv"
pressure = pd.read_csv(pressure_file, index_col = ["name"]).squeeze()

Set up the melt initial FeO prediction model:

In [5]:
wholerock_file = "./data/wholerock.csv"
wholerock = mp.read_melt(wholerock_file, index_col=["name"])

x = wholerock.drop(columns=["FeO"])
FeOi_predict = mpc.FeOi_prediction(x=x, FeO=wholerock["FeO"])

do_not_use = ["MnO", "P2O5", "Cr2O3", "total"]

model_fits = FeOi_predict.calculate_model_fits(exclude=do_not_use)
FeOi_predict.select_predictors(idx=3)

Next, we need to set up the object that handles the random sampling of errors in the Monte Carlo simulation. This is done with the `PEC_MC_parameters` class and it includes the following parameters for error propagation:

- **melt_errors**
        
    propagate errors on melt composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **olivine_errors**

    propagate errors on olivine composition by providing one standard deviation errors per element as a pandas Series (fixed errors for all inclusions) or DataFrame (errors per inclusion).

- **FeOi_errors**

    propagate errors on estimate melt initial FeO contents. Fixed errors can be provided either for the whole dataset, or per inclusion. Alternatively, an [FeOi_prediction object](https://magmapec.readthedocs.io/en/latest/notebooks/FeOi.html) can be provided to propagate errors on predictions models.

- **Fe3Fe2**

    Propagate errors on modelled melt Fe<sup>2+</sup>/Fe<sup>3+</sup> ratios. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

- **Kd**

    Propagate errors on modelled olivine-melt Fe-Mg partition coefficients. Errors are automatically calculated by MagmaPandas based on the selected model. Pass *True* to this parameter to activate it.

By default errors are not propagated - you explicitely need to tell MagmaPEC to do so when initialising the `PEC_MC_parameters` object

In this example we will use all error propagation options, which means we need to provide melt and olivine composition errors. We import these from .csv files containing error data for individual inclusions and olivines. This is just an example with randomly generated errors, normally you should use analytical errors.

In [6]:
melt_errors_file = "./data/melt_errors.csv"
olivine_errors_file = "./data/olivine_errors.csv"

melt_errors = pd.read_csv(melt_errors_file, index_col=[0])
olivine_errors = pd.read_csv(olivine_errors_file, index_col=[0])

Make very sure that the elements in the error data have identical sorting to the melt and olivine dataframes, otherwise errors will be applied to the wrong elements. 
We can force this by sorting the columns of the error dataframes (or series) with the *elements* attributes of the melt and olivine MagmaFrames:

In [7]:
melt_errors = melt_errors[melt.elements]
olivine_errors = olivine_errors[olivine.elements]

Here's what they look like:

In [8]:
melt_errors.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
PI032-04-01,1.02,0.64,0.14,0.47,0.69,0.10,0.08,0.05,0.18,0.05,0.05,0.17,0.21,0.22,0.10,0.12
PI032-04-02,1.06,0.84,0.29,0.46,0.54,0.05,0.03,0.01,0.06,0.18,0.04,0.24,0.12,0.13,0.12,0.04
PI041-02-02,1.04,0.90,0.09,0.55,0.49,0.22,0.21,0.01,0.16,0.04,0.00,0.15,0.23,0.04,0.17,0.06
PI041-03-01,0.98,0.64,0.10,0.40,0.53,0.12,0.12,0.08,0.01,0.03,0.00,0.05,0.36,0.06,0.14,0.04
PI041-03-03,1.02,0.54,0.31,0.42,0.68,0.18,0.03,0.02,0.11,0.06,0.10,0.18,0.08,0.13,0.06,0.15


In [9]:
olivine_errors.head()

,SiO2,FeO,MgO,NiO,MnO,Al2O3,CaO
PI032-04-01,1.06,0.60,1.22,0.01,0.09,0.08,0.02
PI032-04-02,0.85,0.57,1.28,0.14,0.07,0.24,0.06
PI041-02-02,1.11,0.44,1.30,0.08,0.10,0.01,0.19
PI041-03-01,0.99,0.50,1.31,0.14,0.16,0.15,0.01
PI041-03-03,0.95,0.64,1.21,0.13,0.11,0.10,0.15


Together with the `FeOi_prediction` object, we pass these as arguments to the `PEC_MC_parameters` object. We also set *Fe3Fe2* and *Kd* to *True* in order to propagate their model errors.

In [10]:
mc_parameters = mpc.PEC_MC_parameters(melt_errors=melt_errors, olivine_errors=olivine_errors, FeOi_errors=FeOi_predict, Fe3Fe2=True, Kd=True, temperature=True)

Now we can create the Monte Carlo model with the `PEC_MC` object

In [11]:
pec_mc_model = mpc.PEC_MC(inclusions=melt, olivines=olivine, P_bar=pressure, FeO_target=FeOi_predict, MC_parameters=mc_parameters)

In [12]:
pec_mc_model.run(n=100)

Monte Carlo iterations... |█████████████████████████| 100% [100/100] in 5:11.5 


Results are stored internally in the following attributes:

- `pec`: pandas DataFrame

    Average PEC extents (%) of the MC model and their one standard deviation errors. Positive values indicate post-entrapment crystallisation and negative melting.

- `inclusions_corr`: MagmaPandas Melt frame

    Averages of corrected melt inclusion compositions (wt. %)

- `inclusions_stddev`: pandas DataFrame

    One standard deviation errors on inclusions_corr (wt. %)

- `pec_MC`: pandas DataFrame

    PEC extents for individual iterations. Positive values indicate post-entrapment crystallisation and negative melting.

- `inclusions_MC`: dictionary of MagmaPandas Melt frames

    corrected melt inclusion compositions for individual iterations.

In [13]:
pec = pec_mc_model.pec
inclusions_corrected = pec_mc_model.inclusions_corr
inclusions_errors = pec_mc_model.inclusions_stddev

pec_mc = pec_mc_model.pec_MC
inclusions_MC = pec_mc_model.inclusions_MC

In [14]:
pec

,pec,stddev
name,,
PI032-04-01,10.204377,4.297879
PI032-04-02,12.119766,4.459000
PI041-02-02,0.568293,3.069943
PI041-03-01,13.628258,4.989434
PI041-03-03,13.186568,5.253627
PI041-05-04,-3.898197,2.601547
PI041-05-06,2.549099,3.207206
PI041-07-01,12.881526,4.500372
PI041-07-02,12.112288,4.824036


In [15]:
inclusions_corrected

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl
name,,,,,,,,,,,,,,,,
PI032-04-01,48.715442,14.059877,7.822186,9.676603,10.302061,3.585400,0.689181,0.142001,2.436587,0.276075,0.000000,0.592399,1.362597,0.124421,0.149534,0.065637
PI032-04-02,48.387351,14.441329,7.870689,9.335435,10.400122,3.419532,0.902427,0.142546,2.537062,0.316573,0.000000,0.669058,1.282016,0.089072,0.159637,0.047153
PI041-02-02,49.052888,16.796799,5.009636,9.326470,10.186997,3.772607,1.046557,0.153262,2.775611,0.553115,0.000000,0.461175,0.650101,0.048193,0.126220,0.040368
PI041-03-01,45.916543,15.507157,7.479658,10.844116,10.738671,3.300324,1.122307,0.130893,3.110298,0.544042,0.000000,0.782459,0.292353,0.075877,0.101109,0.054192
PI041-03-03,45.129448,15.804634,7.305833,11.125273,10.821364,3.415153,1.163023,0.092606,3.245712,0.503821,0.000000,0.803422,0.315228,0.101197,0.078715,0.094570
PI041-05-04,47.837178,18.615587,3.761804,9.468182,9.310906,4.656684,1.637772,0.146987,2.483599,0.827892,0.000000,0.520125,0.451430,0.099833,0.122007,0.060015
PI041-05-06,46.431548,17.013211,4.856858,8.838603,11.436691,4.024704,1.424116,0.189745,3.623372,0.625452,0.000000,0.646082,0.582644,0.102438,0.139425,0.065112
PI041-07-01,45.667030,15.163162,7.478311,9.682202,11.580323,3.124832,1.262681,0.147222,3.505196,0.553952,0.000000,0.472393,1.019535,0.100693,0.161025,0.081445
PI041-07-02,45.867531,15.447485,7.143116,9.995235,11.483241,3.160580,1.356115,0.133528,3.516759,0.613336,0.000000,0.382312,0.617122,0.073186,0.156535,0.053920


In [16]:
inclusions_errors

,SiO2_stddev,Al2O3_stddev,MgO_stddev,CaO_stddev,FeO_stddev,Na2O_stddev,K2O_stddev,MnO_stddev,TiO2_stddev,P2O5_stddev,Cr2O3_stddev,CO2_stddev,H2O_stddev,F_stddev,S_stddev,Cl_stddev
name,,,,,,,,,,,,,,,,
PI032-04-01,0.691226,0.771084,1.326002,0.566220,0.390037,0.169489,0.077674,0.051972,0.176130,0.041457,0.00000,0.163657,0.185199,0.146397,0.088516,0.078576
PI032-04-02,0.632722,0.920503,1.356268,0.512822,0.269571,0.140494,0.035129,0.013793,0.097578,0.148063,0.00000,0.268646,0.108996,0.088271,0.095708,0.032059
PI041-02-02,0.722465,0.870850,0.962829,0.590430,0.355118,0.233997,0.250135,0.015884,0.176353,0.047586,0.00000,0.151558,0.237201,0.033369,0.134739,0.051186
PI041-03-01,0.596043,0.820049,1.516655,0.531790,0.189291,0.174846,0.106279,0.069043,0.130363,0.036740,0.00000,0.054280,0.244422,0.050157,0.085500,0.034980
PI041-03-03,0.601279,0.775585,1.559648,0.565422,0.243558,0.245323,0.052783,0.024088,0.168359,0.054458,0.00000,0.160780,0.075747,0.109859,0.049107,0.101400
PI041-05-04,0.665418,0.805828,0.800451,0.418976,0.266471,0.201322,0.141601,0.124300,0.088374,0.031545,0.00000,0.221997,0.167583,0.101231,0.011690,0.010031
PI041-05-06,0.914790,1.064496,0.926291,0.393237,0.356566,0.392192,0.258468,0.174935,0.173646,0.051595,0.00000,0.195930,0.290055,0.092081,0.057134,0.026910
PI041-07-01,0.601268,0.900639,1.327086,0.473808,0.244013,0.137994,0.050221,0.067485,0.143328,0.025848,0.00000,0.049513,0.159169,0.090612,0.028369,0.097546
PI041-07-02,0.631810,0.952888,1.405238,0.472917,0.240133,0.185361,0.160311,0.056539,0.147940,0.027124,0.00000,0.175022,0.169111,0.002956,0.050792,0.056534


In [17]:
pec_mc.head()

name,PI032-04-01,PI032-04-02,PI041-02-02,PI041-03-01,PI041-03-03,PI041-05-04,PI041-05-06,PI041-07-01,PI041-07-02,PI052-01-02
iteration,,,,,,,,,,
0,9.133301,11.487427,2.780383,15.350098,15.141699,-2.880859,2.756848,13.958923,14.044922,-5.677368
1,6.0,7.458105,-2.449927,8.190302,9.0401,-6.834772,1.019865,9.391724,7.801364,-8.974222
2,4.544421,8.2,-2.311401,8.899829,5.227661,-5.6,-0.228668,7.197607,4.79444,-9.144247
3,11.588135,11.871948,0.388013,15.496143,15.96001,-5.204565,4.158154,13.291138,13.703125,-8.027454
4,18.889404,24.356006,5.404272,25.192505,20.13877,2.09082,6.503784,21.560791,21.72373,-2.14209


The dataframes in `inclusions_MC` also have *isothermal_equilibration*, *Kd_equilibration*, and *FeO_converge* columns. These columns show if equilibrations during stage [1] and [2] and FeO convergence in stage [2] were successful. Extreme cases of random error sampling may yield melt-olivine pairs that cannot be equilibrated without requiring crystallising exceeding the mass of the inclusion inclusion or exchange of more Mg or Fe than the inclusion contains. If that is the case, no corrected compositions are calculated and the isothermal- of Kd-equilibration column is set to *False*.

In [18]:
inclusions_MC["PI032-04-01"].head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,Cr2O3,CO2,H2O,F,S,Cl,isothermal_equilibration,Kd_equilibration,FeO_converge
iteration,,,,,,,,,,,,,,,,,,,
0,48.67285,13.888554,7.448828,9.97345,10.156829,3.683243,0.582864,0.131517,2.387913,0.270306,0.0,0.792358,1.516282,0.409977,0.056141,0.028889,True,True,True
1,49.339541,14.945395,6.260236,9.707224,9.986188,3.76925,0.635161,0.205038,2.428781,0.317446,0.0,0.43033,1.46976,0.260401,0.245252,0.0,True,True,True
2,49.185887,14.657787,6.191133,10.536487,9.69496,3.481325,0.811659,0.128366,2.343293,0.264723,0.0,0.724772,1.479573,0.0,0.249312,0.250721,True,True,True
3,47.611332,13.210966,9.0248,9.843957,10.55474,3.554355,0.777219,0.037187,2.604992,0.24101,0.0,0.587772,1.311468,0.353683,0.230626,0.055893,True,True,True
4,48.381508,13.202514,10.792778,8.756421,10.302342,3.365715,0.621507,0.192857,2.018364,0.296873,0.0,0.414829,1.300231,0.165467,0.188594,0.0,True,True,True
